In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import numpy as np
import math
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import jaccard_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_auc_score
from sklearn.metrics import mean_squared_error
from scipy import stats
from sklearn.impute import SimpleImputer
import warnings 
from warnings import simplefilter
# ignore all types of warnings
pd.options.mode.chained_assignment = None 
simplefilter(action='ignore', category=UserWarning) 
simplefilter(action='ignore', category=FutureWarning) 
#warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', None)
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
## %%time
train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')#,nrows = 1*(10**5))
#train.shape
#train = train[train['date'] > 85]
#train = train[train['weight'] > 0]
#train['action'] = (train ['resp'] > 0).astype(int)
#train['action'] =  (train['weight'] < 3) & ((train['resp_1'] > 0 ) | (train['resp_2'] > 0 ) | (train['resp_3'] > 0 )\
 #                   | (train['resp_4'] > 0) | (train['resp'] > 0 ) ).astype('int')

In [ ]:
train['resp_1'] = train['resp_1']*train['weight']
train['resp_2'] = train['resp_2']*train['weight']
train['resp_3'] = train['resp_3']*train['weight']
train['resp_4'] = train['resp_4']*train['weight']
train['resp'] = train['resp']*train['weight']

In [ ]:
# train = pd.read_csv('../input/jane-street-market-prediction/train.csv')
#train = train.query('date > 85').reset_index(drop = True) 
train = train[train['date'] > 85]
#train = train[train['date'] != 161]
#train = train[train['date'] != 168]
#train = train[train['date'] != 270]
#train = train[train['date'] != 294]
#train = train[train['date'] != 459]
train = train[train['weight'] != 0]
features = [c for c in train.columns if "feature" in c]
features.remove('feature_0')
train.fillna(train[features].mean(),inplace=True)
train['action'] = ((train['resp'].values) > 0).astype(int)

In [ ]:

f_mean = np.mean(train[features].values,axis=0)
#fmean = np.mean(train[features].ewm(com=5,axis=0).mean())
#f_mean = fmean.values
print (f_mean.shape)
#print(f_mean[:5])

resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']

#X_train = train.loc[:, train.columns.str.contains('feature')]
X_train=train[features].values
#y_train = (train.loc[:, 'action'])

y_train = (np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T)
print (y_train[:,4])

In [ ]:
import lightgbm as lgbm
from lightgbm import LGBMClassifier
from sklearn.model_selection import train_test_split
# modeling step 
models = [] # list of model , we will train
#th = 0.50000
#model_acc = 0
params={"num_leaves":300,
       "max_bin":450,
       "feature_fraction":0.52,
       "bagging_fraction":0.52,
       "objective":"binary",
       "learning_rate": 0.05,
       "boosting_type":"gbdt",
       "metric":"auc"
       }

for i in range(y_train.shape[1]):
    xtr,xval,ytr,yval = train_test_split(X_train ,y_train[:,i],test_size=0.2,
                                                 stratify=y_train[:,i],)
   
    d_train = lgbm.Dataset(xtr,label=ytr)
    d_eval = lgbm.Dataset(xval,label=yval,reference=d_train)
    clf = lgbm.train(params,d_train,valid_sets=[d_train,d_eval],num_boost_round=1000,\
    early_stopping_rounds=50,verbose_eval=50)
    models.append(clf)
    #break
    #preds = clf.predict(xval) #np.mean[model.predict(xval) for model in models]
    #preds = np.where(preds >= th, 1, 0).astype(int)
    #pred_labels = np.rint(preds)
    #accuracy = sklearn.metrics.accuracy_score(yval, preds)
    #print ('Learning_rate {}'.format(lr),accuracy)
    #if accuracy > model_acc:
     #   model_acc = accuracy
      #  Bst_lr = lr
#print ('Final Accuracy is {} with best learning rate {} '.format(model_acc,Bst_lr))


lbl = LabelEncoder()
lbl.fit(list(train['action'].values) + list(train['action'].values))
train['action'] = lbl.transform(list(train['action'].values))
    

#f_mean = np.mean(train[features[1:]].values,axis=0)

resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']

#X_train = train.loc[:, train.columns.str.contains('feature')]
#X_train=train[features].values
#y_train = (train.loc[:, 'action'])

z_train = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
z_train

features = [c for c in train.columns if "feature" in c]
features.remove('feature_0')
f_mean = np.mean(train[features].values,axis=0)

In [ ]:
#features.append('date')

In [ ]:
#features

In [ ]:
#train['action'] = train.apply(lambda x: action_value(x['weight']),axis=1)

lst = []
for i in range(130):
    lst.append('feature_{}'.format(i))
print (lst)

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.feature_selection import VarianceThreshold
from sklearn import preprocessing
import lightgbm as lgbm
from lightgbm import LGBMClassifier
sk_fold = StratifiedKFold(n_splits=5,random_state=42,shuffle=True)
resp_cols = ['resp_1','resp_2','resp_3','resp_4','resp']
models = []
jc_score = []

params={"num_leaves":300,
       "max_bin":450,
       "feature_fraction":0.52,
       "bagging_fraction":0.52,
       "objective":"binary",
       "learning_rate":0.05,
       "boosting_type":"gbdt",
       "metric":"auc"
       }
for col in resp_cols:
    train = pd.read_csv('/kaggle/input/jane-street-market-prediction/train.csv')#,\
                        #nrows = 1*(10**5))
    train = train[train['date'] > 85]
    train = train[train['weight'] > 0]
    train['action'] = (train [[col]] > 0).astype(int)
    lbl = LabelEncoder()
    lbl.fit(list(train['action'].values) + list(train['action'].values))
    train['action'] = lbl.transform(list(train['action'].values))
    #X = train.iloc[:,8:-2]
    X = train[features].values
#print (X.shape)
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    X = imp_mean.fit(X).transform(X)
    X = np.asarray(X)
    X = preprocessing.StandardScaler().fit(X).transform(X)

#resp_cols = ['resp_1', 'resp_2', 'resp_3', 'resp', 'resp_4']
#y = np.stack([(train[c] > 0).astype('int') for c in resp_cols]).T
    y =  np.asarray(train['action'])
    #print ("y",y[:5])
    for train, test in sk_fold.split(X,y):
        X_train, X_test = X[train], X[test]
        y_train, y_test = y[train], y[test]
    print ('Train set:', X_train.shape,  y_train.shape)
    print ('Test set:', X_test.shape,  y_test.shape)
    print ('=====================================')
    print("StratifiedKFold done")
    d_train = lgbm.Dataset(X_train,label=y_train.ravel())
    d_eval = lgbm.Dataset(X_test,label=y_test,reference=d_train)
    model = lgbm.train(params,d_train,valid_sets=[d_train,d_eval],num_boost_round=1000,\
                    early_stopping_rounds=50,verbose_eval=50)
    #jc_score.append((jc,col))
    models.append(model)
    #break

from sklearn.model_selection import GroupKFold
from sklearn import preprocessing
#X = train.iloc[:,8:-2]
X = train[features].values
print (X.shape)
imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
X = imp_mean.fit(X).transform(X)
X = np.asarray(X)
X = preprocessing.StandardScaler().fit(X).transform(X)
y =  np.asarray(train[['action']])
groups = np.array(train['date'])
group_kfold = GroupKFold(n_splits=5)
for train, test in group_kfold.split(X,y,groups):
    X_train, X_test = X[train], X[test]
    y_train, y_test = y[train], y[test]
print ('Train set:', X_train.shape,  y_train.shape)
print ('Test set:', X_test.shape,  y_test.shape)
print ('=====================================')
print("GroupKFold done")

y = y_test[:15218]
y.shape

import tensorflow as tf
from tensorflow import keras
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam,SGD,RMSprop
from sklearn.model_selection import train_test_split

#opt = tf.keras.optimizers.Adam(learning_rate=0.1)
adam = Adam(lr=.001,amsgrad=True)
model = keras.Sequential([
    keras.layers.Flatten(input_shape=(129,)),
    keras.layers.Dense(80, activation=tf.nn.relu),
    tf.keras.layers.Dropout(.1, input_shape=(129,)),
    keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dropout(.1, input_shape=(129,)),
    keras.layers.Dense(1, activation=tf.nn.sigmoid),
])
#model.add(keras.layers.Dropout(0.5))
# compile the model
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['acc'])

# fit the model
history=model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=256,verbose=1)


import matplotlib.pyplot as plt
# list all data in history
print(history.history.keys())
# summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

s = model.predict_classes(X_test)
s[:20]

%%time
LR = LogisticRegression(C=0.01, solver='saga',max_iter=1000,
                                n_jobs=-1, random_state=42).fit(X_train,y_train.ravel()) 
yhat = LR.predict(X_test) 
yhat_prob = LR.predict_proba(X_test)
from sklearn.metrics import jaccard_score
print (round(jaccard_score(y_test, yhat),3)) 

from sklearn.metrics import classification_report 
print (classification_report(y_test, yhat))

from sklearn.metrics import log_loss
log_loss(y_test, yhat_prob)

from sklearn.metrics import classification_report
print (classification_report(y_test, yhat))

In [ ]:
# Recall 0.02 & 0.99 For (weight < 1) & (OR Combination with all resp) accuracy: 0.68
# Recall 0.01 & 0.99 For (weight < 2) & (OR Combination with all resp) accuracy: 0.70
# Recall 0.59 & 0.89 For (weight < 3) & (OR Combination with all resp) accuracy:0.77 GroupKFold

In [ ]:
# Recall 0.38 and 0.76 for weight < 3 accuracy : 0.56 
# Recall 0.48 and 0.63 for weight < 2 accuracy : 0.56 
# Recall 0.63 and 0.49 for weight < 1 accuracy : 0.56 
# Recall 0.36 and 0.65 for resp    accuracy : 0.51 
# Recall 0.41 and 0.61 for resp_3  accuracy : 0.51 



import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

(test_df, sample_prediction_df) = next(iter_test)
test_df

test_df.iloc[:,2:-1]

env.predict(sample_prediction_df)

In [ ]:
%%time
f = np.median
th = 0.5000
lst = []
import janestreet
env = janestreet.make_env()
for (test_df, pred_df) in env.iter_test():
    #if test_df['weight'].item() > 0:
        #x_tt = test_df.loc[:, features].values
    x_tt=test_df[features].values
    if np.isnan(x_tt[:,:].sum()):
        x_tt[:,:] = np.nan_to_num(x_tt[:,:]) + np.isnan(x_tt[:,:]) * f_mean
    pred = np.mean([model.predict(x_tt) for model in models],axis=0)
    pred = f(pred)
    pred_df.action = np.where(pred >= th, 1, 0).astype(int)
    lst.append(pred)
    #else:
     #   pred_df.action = 0
    env.predict(pred_df)

In [ ]:
print(lst)

In [ ]:
#print('AUC:', round(roc_auc_score(y,ypred),3)) # 0.499 for resp_3 and whole data
#print ('JS:', round(jaccard_score(y,ypred),3)) # 0.368 same as above